### Final version Scrapping Sudoku UK


In [1]:
from bs4 import BeautifulSoup
import urllib3
import requests 
import datetime
import numpy as np
import pandas as pd
from urllib.request import urlopen 
from urllib.error import HTTPError 
from urllib.error import URLError
from tqdm import tqdm

## FUNCTIONS

## Get_dates():
To extract all the data we need to open all the different solutions that have been posted during the time in the web site. 

In [27]:
def list_dates():
  dates = []
  dates_error = []
  urls = []
  for i in tqdm(range(2,10)):
    d = datetime.date.today()- datetime.timedelta(days = i)
    d_format = str(d.day)+'/'+str(d.month) +'/'+ str(d.year)
    URL = "http://www.sudoku.org.uk/DailySudoku.asp?solution=please&day="+d_format
    try:
      urlopen(URL)
    except HTTPError as e:
        pass
#       dates_error.append(d_format)
    except URLError as e:
        pass
#       dates_error.append(d_format)
    else:
#       dates.append(d_format)
      urls.append(URL)
  return urls


## Get_html()
Using the list of the days we're going to open each URL and extract all the HTML code.

In [28]:
def get_html(url):
  r = requests.get(url) 
  return BeautifulSoup(r.content,'html.parser')

## Consolidate()
extract all the sudokus and their solutions, the level of difficulty, number of people that solved the soduku and the average time in minutes.

In [29]:
def consolidate(urls):
    solution, sudoku, level, people,av_time, unit = ([] for i in range(6))

    for url in tqdm(urls):
        a, b = ([] for i in range(2))
        soup = get_html(url)
        for link in soup.find_all('td', attrs={'class': ['InnerTDone2','InnerTDone'] }):
            if link.attrs['class'] == ['InnerTDone2']:
              b.append(link.text)
            else:
              b.append('.')
            a.append(link.text)
            
        sudoku.append(''.join(b))
        solution.append(''.join(a))
             
        p = list(list(soup.table.td)[2])
        level.append(str(p[1].get_text()).split(", ")[1].split()[0])
        people.append(str(p[3]).split()[0])
        av_time.append(str(p[3]).split()[6])
        unit.append(str(p[3]).split()[7])
        
    return  urls, level, people, av_time, unit, sudoku, solution 

# Extract Dates

In [30]:
# dates, urls, dates_error = list_dates()
urls = list_dates()

100%|██████████| 8/8 [00:02<00:00,  2.95it/s]


In [31]:
len(urls)

8

# Export URLs to CSV

In [32]:
urls_df = pd.DataFrame(list(urls), 
               columns =['urls'])
urls_df.to_csv('urls_test.csv')

# Load URLs

In [33]:
df_urls = pd.read_csv('urls_test.csv', sep=',').drop('Unnamed: 0', axis=1)

In [34]:
df_urls

,urls
0,http://www.sudoku.org.uk/DailySudoku.asp?solut...
1,http://www.sudoku.org.uk/DailySudoku.asp?solut...
2,http://www.sudoku.org.uk/DailySudoku.asp?solut...
3,http://www.sudoku.org.uk/DailySudoku.asp?solut...
4,http://www.sudoku.org.uk/DailySudoku.asp?solut...
5,http://www.sudoku.org.uk/DailySudoku.asp?solut...
6,http://www.sudoku.org.uk/DailySudoku.asp?solut...
7,http://www.sudoku.org.uk/DailySudoku.asp?solut...


#### before Mar 7 of 2006 location 5010 there are not registers about the players or the average time in the website, for this reason we're cut this urls 

In [35]:
# index = df_urls.index[df_urls['urls'].str.contains('=7/3/2006', regex=False)]

In [36]:
# print(index[0])

In [37]:

# 


In [38]:
# new_urls.shape

In [39]:
# #Export to csv
# new_urls.to_csv('new_urls.csv')

# Extract Data

In [40]:
urls, level, people,av_time, unit, sudoku, solution = consolidate(df_urls['urls'])

100%|██████████| 8/8 [00:02<00:00,  3.06it/s]


# Create Dataframe

In [41]:
df = pd.DataFrame(list(zip(urls, level, people,av_time, unit, sudoku, solution)), 
               columns =['URL', 'Level','People','Average-Time', 'Unit-Time', 'Sudoku', 'Solution']) 

In [42]:
df.head()

,URL,Level,People,Average-Time,Unit-Time,Sudoku,Solution
0,http://www.sudoku.org.uk/DailySudoku.asp?solut...,Gentle,283,12,minutes,..7..34....17528......8.....4.5...3.1.2...5.8....,9871634254317528696259843718495167321623795487...
1,http://www.sudoku.org.uk/DailySudoku.asp?solut...,Diabolical,238,30,minutes,.6...3.7.....6.1.49.......3..97.45..7...1...9....,8641239753259671849175486232897345617462158391...
2,http://www.sudoku.org.uk/DailySudoku.asp?solut...,Moderate,274,14,minutes,.54....8.....4..7.79...8.....26..1....35..2......,1547639823289456717962183459726341588435712965...
3,http://www.sudoku.org.uk/DailySudoku.asp?solut...,Diabolical,244,28,minutes,..2...9......425.....751...4.......63.7...8..8...,5126389477639425819847516234218953763571268948...
4,http://www.sudoku.org.uk/DailySudoku.asp?solut...,Tough,251,27,minutes,23..6...8.9.....5.....4....1....4..3..72.15..5...,2359674184983127566715482391296548733872915645...


In [43]:
df.shape

(8, 7)

In [44]:
len(df['Sudoku'].value_counts())

8

In [45]:
len(df['Solution'].value_counts())

8

## Splititng DataFrame 
#### DF1: URL,Level, unit-time,  and solution 
#### DF2: People, Average-Time and Sudoku


In [46]:
df['Id'] = df.index

In [47]:
df.head()

,URL,Level,People,Average-Time,Unit-Time,Sudoku,Solution,Id
0,http://www.sudoku.org.uk/DailySudoku.asp?solut...,Gentle,283,12,minutes,..7..34....17528......8.....4.5...3.1.2...5.8....,9871634254317528696259843718495167321623795487...,0
1,http://www.sudoku.org.uk/DailySudoku.asp?solut...,Diabolical,238,30,minutes,.6...3.7.....6.1.49.......3..97.45..7...1...9....,8641239753259671849175486232897345617462158391...,1
2,http://www.sudoku.org.uk/DailySudoku.asp?solut...,Moderate,274,14,minutes,.54....8.....4..7.79...8.....26..1....35..2......,1547639823289456717962183459726341588435712965...,2
3,http://www.sudoku.org.uk/DailySudoku.asp?solut...,Diabolical,244,28,minutes,..2...9......425.....751...4.......63.7...8..8...,5126389477639425819847516234218953763571268948...,3
4,http://www.sudoku.org.uk/DailySudoku.asp?solut...,Tough,251,27,minutes,23..6...8.9.....5.....4....1....4..3..72.15..5...,2359674184983127566715482391296548733872915645...,4


In [37]:
df1 = df[['Id','URL', 'Level','Unit-Time','Solution']]
df2 = df[['Id','People','Average-Time','Sudoku']]

In [39]:
df1.shape, df2.shape

((5014, 5), (5014, 4))

In [40]:
df1.head()

,Id,URL,Level,Unit-Time,Solution
0,0,http://www.sudoku.org.uk/DailySudoku.asp?solut...,Moderate,minutes,1547639823289456717962183459726341588435712965...
1,1,http://www.sudoku.org.uk/DailySudoku.asp?solut...,Diabolical,minutes,1547639823289456717962183459726341588435712965...
2,2,http://www.sudoku.org.uk/DailySudoku.asp?solut...,Tough,minutes,1547639823289456717962183459726341588435712965...
3,3,http://www.sudoku.org.uk/DailySudoku.asp?solut...,Moderate,minutes,1547639823289456717962183459726341588435712965...
4,4,http://www.sudoku.org.uk/DailySudoku.asp?solut...,Moderate,minutes,1547639823289456717962183459726341588435712965...


In [41]:
df2.head()

,Id,People,Average-Time,Sudoku
0,0,274,14,.54....8.....4..7.79...8.....26..1....35..2......
1,1,244,28,.54....8.....4..7.79...8.....26..1....35..2......
2,2,251,27,.54....8.....4..7.79...8.....26..1....35..2......
3,3,264,15,.54....8.....4..7.79...8.....26..1....35..2......
4,4,285,16,.54....8.....4..7.79...8.....26..1....35..2......


# Function to visualize Sudoku

In [23]:
def split(element): 
    return [char for char in element]  

def transform_matrix(element):
  return np.reshape(split(element), (-1, 9))

In [48]:
transform_matrix(df.Solution[0])

array([['9', '8', '7', '1', '6', '3', '4', '2', '5'],
       ['4', '3', '1', '7', '5', '2', '8', '6', '9'],
       ['6', '2', '5', '9', '8', '4', '3', '7', '1'],
       ['8', '4', '9', '5', '1', '6', '7', '3', '2'],
       ['1', '6', '2', '3', '7', '9', '5', '4', '8'],
       ['7', '5', '3', '4', '2', '8', '9', '1', '6'],
       ['2', '9', '4', '6', '3', '5', '1', '8', '7'],
       ['3', '7', '6', '8', '9', '1', '2', '5', '4'],
       ['5', '1', '8', '2', '4', '7', '6', '9', '3']], dtype='<U1')

In [49]:
transform_matrix(df.Sudoku[0])

array([['.', '.', '7', '.', '.', '3', '4', '.', '.'],
       ['.', '.', '1', '7', '5', '2', '8', '.', '.'],
       ['.', '.', '.', '.', '8', '.', '.', '.', '.'],
       ['.', '4', '.', '5', '.', '.', '.', '3', '.'],
       ['1', '.', '2', '.', '.', '.', '5', '.', '8'],
       ['.', '5', '.', '.', '.', '8', '.', '1', '.'],
       ['.', '.', '.', '.', '3', '.', '.', '.', '.'],
       ['.', '.', '6', '8', '9', '1', '2', '.', '.'],
       ['.', '.', '8', '2', '.', '.', '6', '.', '.']], dtype='<U1')

In [50]:
transform_matrix(df.Sudoku[1])

array([['.', '6', '.', '.', '.', '3', '.', '7', '.'],
       ['.', '.', '.', '.', '6', '.', '1', '.', '4'],
       ['9', '.', '.', '.', '.', '.', '.', '.', '3'],
       ['.', '.', '9', '7', '.', '4', '5', '.', '.'],
       ['7', '.', '.', '.', '1', '.', '.', '.', '9'],
       ['.', '.', '3', '8', '.', '6', '2', '.', '.'],
       ['4', '.', '.', '.', '.', '.', '.', '.', '2'],
       ['5', '.', '1', '.', '8', '.', '.', '.', '.'],
       ['.', '3', '.', '4', '.', '.', '.', '1', '.']], dtype='<U1')

# Export Data Frame

In [22]:
df.to_csv('sudoku_uk_test.csv')

In [42]:
df1.to_csv('df1.csv')
df2.to_csv('df2.csv')